In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import numpy as np

In [4]:
import cloudknot as ck

In [5]:
ck.set_region('us-west-2')

In [6]:
s3_access_pars = ck.Pars(name='s3_access',
                         policies=('AmazonS3FullAccess',),
                         use_default_vpc=False)

In [7]:
import boto3

In [8]:

client = boto3.resource('s3')
bucket_name = 'arokem.mri2mri'
b = client.Bucket(bucket_name)


In [9]:
T1_list = [ll.key for ll in list(b.objects.all())[1:] if (ll.key.startswith("IXI-data/multimodal_registration/") and ll.key.endswith('T1.nii.gz') and "myunet" in ll.key)]

In [10]:
t1_path = T1_list[0]

In [11]:
def compare_registrations(t1_path):
    import os.path as op

    import boto3
    import numpy as np
    from AFQ import registration as reg
    import nibabel as nib
    from dipy.core.geometry import decompose_matrix
    
    client = boto3.resource('s3')
    bucket_name = 'arokem.mri2mri'
    b = client.Bucket(bucket_name)

    data_path = op.split(t1_path)[0]    
    source_string = 'DTI-00_transformed.nii.gz'
    
    print("Downloading %s as target"%t1_path)
    b.download_file(t1_path, 'target.nii.gz')
    img_target = nib.load('target.nii.gz')
    source_path = op.join(data_path, op.split(t1_path)[1].split('T1')[0] + source_string)
    print("Downloading %s as source"%source_path)
    b.download_file(source_path, 'source.nii.gz')
    img_source = nib.load('source.nii.gz')
    pred_path = op.join(data_path,  op.split(t1_path)[1].split('T1')[0]+ 'T1_transformed_predict.nii.gz')
    print("Downloading %s as prediction"%pred_path)
    b.download_file(pred_path, 'pred.nii.gz')
    img_pred = nib.load('pred.nii.gz')

    data_source = img_source.get_data()
    data_target = img_target.get_data()
    data_pred = img_pred.get_data()
    print("Registering source to target")
    transformed_source, reg_affine_source = reg.affine_registration(moving=data_source, 
                                                                static=data_target, 
                                                                moving_affine=np.eye(4),
                                                                static_affine=np.eye(4),
                                                                level_iters=[10000, 1000, 100])    

    print("Registering prediction to target")
    transformed_pred, reg_affine_pred = reg.affine_registration(moving=data_pred, 
                                                                static=data_target, 
                                                                moving_affine=np.eye(4),
                                                                static_affine=np.eye(4),
                                                                level_iters=[10000, 1000, 100])    

    
    print("Calculating registration errors")
    true_transform = np.dot(np.linalg.inv(img_target.affine), img_source.affine)

    scale_true, shear_true, angles_true, translate_true, perspective_true = decompose_matrix(true_transform)
    scale_pred, shear_pred, angles_pred, translate_pred, perspective_pred = decompose_matrix(np.linalg.inv(reg_affine_pred))
    scale_source, shear_source, angles_source, translate_source, perspective_source = decompose_matrix(np.linalg.inv(reg_affine_source))
    
    translate_true - translate_source 
    this_RMS_trans_source = np.mean((np.array(translate_true)- 
                                     np.array(translate_source)) ** 2)
    this_RMS_trans_pred = np.mean((np.array(translate_true) - 
                                   np.array(translate_pred)) ** 2)
    this_RMS_rot_source = np.mean((np.array(angles_true) - 
                                   np.array(angles_source)) ** 2)
    this_RMS_rot_pred = np.mean((np.array(angles_true) - 
                                 np.array(angles_pred)) ** 2)

    
    results_file = open('results_%s.txt'%op.split(t1_path)[-1], 'w')
    print("Writing results to %s"%results_file)
    results_file.write('%2.8f\t %2.8f\t %2.8f\t %2.8f\t' % (this_RMS_trans_pred, 
                                                            this_RMS_rot_pred,
                                                            this_RMS_trans_source,
                                                            this_RMS_rot_source))
    results_file.close()
    upload_file = op.join(data_path, results_file.name)
    print("Uploading results to %s" % upload_file)
    b.upload_file(results_file.name, upload_file)

In [12]:
#compare_registrations(T1_list[0])

In [13]:
image = ck.DockerImage(func=compare_registrations, github_installs="https://github.com/yeatmanlab/pyAFQ.git")

In [14]:
image.docker_path

'/Users/arokem/projects/mri2mri/cloudknot_docker_compare_registrations_9_sai54g/Dockerfile'

In [15]:
image.build(tags='0.6')

In [16]:
image.images

[{'name': 'cloudknot/compare_registrations', 'tag': '0.6'}]

In [17]:
reg_knot = ck.Knot(name='compare_registrations',
                   docker_image=image,
                   pars=s3_access_pars, 
                   resource_type="SPOT", 
                   bid_percentage=100)

In [18]:
#reg_knot.clobber()

In [19]:
result = reg_knot.map(T1_list)

In [20]:
j0 = reg_knot.jobs[0]

In [42]:
j0.status

{'status': 'SUCCEEDED',
 'statusReason': None,
 'attempts': [],
 'arrayProperties': {'statusSummary': {'STARTING': 0,
   'FAILED': 0,
   'RUNNING': 0,
   'SUCCEEDED': 395,
   'RUNNABLE': 0,
   'SUBMITTED': 0,
   'PENDING': 0},
  'size': 395}}

In [43]:
reg_knot.clobber()